<a href="https://colab.research.google.com/github/Disaster4255/PL_REPO_PY_Hao/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80_HW1%262.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 預算管理系統 程式語言HW1&2

##使用方法
使用者頁面:<br>
輸入使用者名稱、要購買的品項、該品項屬於哪種類別、該品項的單價、欲購買的數量，完成輸入後可選擇分析該物品單價與市場的差異並顯示在下方。完成後按下送出即可將購買紀錄計入預算統計內。<br>
最下方可以刷新所有採購紀錄<br>
<br>
管理員頁面:<br>
如要使用管理員頁面，請先在密碼欄位輸入"disaster4255"再執行同頁面的其他操作

In [ ]:
# 1. 安裝 Gradio
!pip install -q gradio


In [ ]:
pip install -q -U google-genai

In [25]:
# 1. Gemini API 設定
from google import genai
from google.genai import types
import gradio as gr

# ⚠️ 確保從 Colab Secrets 讀取您設定的金鑰
try:
    from google.colab import userdata
    # 這裡假設您的金鑰 Secret 名稱就是 GEMINI_API_KEY (標準命名)
    # 如果您設定了不同的名稱，請替換下面的字串
    API_KEY = userdata.get('gemini_key')

    # 使用讀取到的金鑰來初始化 Client
    client = genai.Client(api_key=API_KEY)
    print("✅ Gemini Client 初始化成功，已使用 Colab Secret 金鑰。")
except Exception as e:
    # 確保在非 Colab 環境下也能執行 (例如本地電腦)
    client = genai.Client()
    print(f"⚠️ 無法從 Colab Secrets 讀取金鑰，嘗試使用環境變數或預設配置初始化。錯誤: {e}")

MODEL = 'gemini-2.5-flash'

# 2. Google Sheet 連線設定
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
import pandas as pd
from datetime import datetime

SHEET_URL = 'https://docs.google.com/spreadsheets/d/1HEtQ5xUKfOXhpKNrqwBfg-BwR__4D3FgBfVLLlKIChw/edit?usp=sharing'
creds, _ = default()
gc = gspread.authorize(creds)
sh = gc.open_by_url(SHEET_URL)
config_ws = sh.worksheet('設定表')
record_ws = sh.worksheet('支出紀錄')
ADMIN_PASSWORD = "disaster4255"

# 3. 資料處理函式
def get_budget_data():
    data = config_ws.get_all_records()
    return {row["分類"]: {"上限": row["預算上限"], "已花": row["已花金額"]} for row in data}

def get_category_list():
    return list(get_budget_data().keys())

def update_spent(category, new_spent):
    data = config_ws.get_all_records()
    for i, row in enumerate(data, start=2):
        if row["分類"] == category:
            config_ws.update_cell(i, 3, new_spent)
            break

def add_or_update_category(category, limit):
    data = config_ws.get_all_records()
    found = False
    for i, row in enumerate(data, start=2):
        if row["分類"] == category:
            config_ws.update_cell(i, 2, limit)
            found = True
            break
    if not found:
        config_ws.append_row([category, limit, 0])

def add_record(user, item, category, price, qty):
    total = price * qty
    record_ws.append_row([
        datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        user, item, category, price, qty, total
    ])

def recalc_spent():
    records = record_ws.get_all_records()
    budgets = get_budget_data()
    spent_totals = {k: 0 for k in budgets.keys()}
    for r in records:
        cat = r["分類"]
        if cat in spent_totals:
            spent_totals[cat] += r["小計"]
    for i, row in enumerate(config_ws.get_all_records(), start=2):
        cat = row["分類"]
        config_ws.update_cell(i, 3, spent_totals.get(cat, 0))

def get_record_df():
    data = record_ws.get_all_records()
    if not data:
        return pd.DataFrame()
    df = pd.DataFrame(data)
    return df

# 4. Gemini 價格分析函式 (新增)
def analyze_price(item: str, price: str, category: str) -> str:
    """使用 Gemini 分析輸入單價是否合理"""
    try:
        # 轉換價格為浮點數，並確保品項不為空
        item = item.strip()
        price_float = float(price)

        if not item:
            return "❌ **請輸入購買品項名稱**以進行價格分析。"

        # 建立提示詞 (Prompt)
        prompt = f"""
        請你扮演一個經驗豐富的市場分析師。
        使用者在預算管理程式中輸入了一筆消費紀錄：
        - **品項**：{item}
        - **單價**：{price_float} 元 (新台幣)
        - **類別**：{category}

        請根據你的市場知識，**簡潔地**分析這個單價是否在合理的市場價格範圍內。

        請以中文回答，並根據以下規則輸出：
        1. 如果價格看起來合理，請以「✅ **價格看起來合理**」開頭，然後加上簡短的解釋。
        2. 如果價格看起來偏高，請以「⚠️ **價格可能偏高**」開頭，然後加上簡短的建議，例如建議在哪裡可以找到更便宜的價格，或說明合理的價格範圍。
        3. 如果價格看起來偏低，請以「⭐ **撿到便宜了！**」開頭，然後給予鼓勵。
        4. 如果品項名稱太模糊（例如：食物、東西），請說明需要更具體的品項名稱。

        請勿輸出任何數學公式或計算過程。
        """

        # 呼叫 Gemini API
        response = client.models.generate_content(
            model=MODEL,
            contents=prompt
        )
        return response.text

    except ValueError:
        return "❌ **單價輸入錯誤**，請輸入有效的數字。"
    except Exception as e:
        print(f"Gemini API 呼叫錯誤: {e}")
        return "❌ **價格分析服務暫時無法使用**。"

# 5. Gradio 事件函式 (user_submit 保持不變)
def admin_view(password):
    if password != ADMIN_PASSWORD:
        return gr.Markdown("❌ 密碼錯誤")
    budgets = get_budget_data()
    msg = "### 📊 目前分類與上限\n"
    for c, v in budgets.items():
        msg += f"- {c}：上限 {v['上限']} 元，已花 {v['已花']} 元，剩餘 {v['上限']-v['已花']} 元<br>"
    return gr.Markdown(msg)

def admin_add(password, category, limit):
    if password != ADMIN_PASSWORD:
        return gr.Markdown("❌ 密碼錯誤")
    try:
        limit = float(limit)
        add_or_update_category(category, limit)
        return gr.Markdown(f"✅ 已更新/新增分類「{category}」上限為 {limit} 元。")
    except:
        return gr.Markdown("❌ 請輸入有效數字")

def admin_recalc(password):
    if password != ADMIN_PASSWORD:
        return gr.Markdown("❌ 密碼錯誤")
    recalc_spent()
    return gr.Markdown("🔄 已重新計算所有分類支出。")

def user_submit(user, item, category, price, qty):
    try:
        price = float(price)
        qty = float(qty)
        budgets = get_budget_data()
        if category not in budgets:
            return gr.Markdown("❌ 沒有這個分類")
        spent, limit = budgets[category]["已花"], budgets[category]["上限"]
        total = price * qty
        if spent + total > limit:
            return gr.Markdown(f"💸 預算不足！剩餘 {limit-spent} 元")
        add_record(user, item, category, price, qty)
        update_spent(category, spent + total)
        return gr.Markdown(f"✅ 登記成功！目前分類「{category}」已花 {spent + total}/{limit} 元")
    except:
        return gr.Markdown("❌ 請輸入有效的數字")

def user_list():
    df = get_record_df()
    if df.empty:
        return gr.Markdown("目前尚無支出紀錄。")
    return gr.Dataframe(df)

def update_category_dropdown():
    categories = get_category_list()
    return gr.Dropdown.update(choices=categories)

# 6. Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🧾 支出管理系統（Google Sheets）")



    with gr.Tab("使用者登記"):
        with gr.Row():
            usr = gr.Textbox(label="名字")
            itm = gr.Textbox(label="購買品項")
        with gr.Row():
            # Dropdown分類，右方提供強制更新分類按鈕
            cat_dropdown = gr.Dropdown(label="分類", choices=get_category_list())
            update_cat_btn = gr.Button("更新分類")
            update_cat_btn.click(update_category_dropdown, outputs=cat_dropdown)
        with gr.Row():
            price = gr.Textbox(label="單價")
            qty = gr.Textbox(label="數量")

        # 新增：價格分析按鈕與輸出
        analyze_btn = gr.Button("💰 分析價格合理性")
        price_analysis_output = gr.Markdown("**在此顯示價格合理性分析結果...**")

        # 設定價格分析按鈕點擊事件
        analyze_btn.click(
            analyze_price,
            inputs=[itm, price, cat_dropdown], # 傳入 品項、單價、分類
            outputs=price_analysis_output # 輸出到分析結果框
        )

        submit_btn = gr.Button("送出")
        user_output = gr.Markdown()
        submit_btn.click(user_submit, [usr, itm, cat_dropdown, price, qty], user_output)
        gr.Markdown("---")
        gr.Markdown("### 所有支出紀錄")
        list_btn = gr.Button("更新支出紀錄")
        data_output = gr.Dataframe()
        list_btn.click(user_list, [], [data_output])

    with gr.Tab("管理員模式"):
        with gr.Row():
            pwd = gr.Textbox(type="password", label="管理員密碼", value="")
        with gr.Row():
            view_btn = gr.Button("查看全部分類")
            view_output = gr.Markdown()
            view_btn.click(admin_view, [pwd], view_output)
        gr.Markdown("---")
        with gr.Row():
            cat = gr.Textbox(label="分類名稱")
            lim = gr.Textbox(label="預算上限")
            add_btn = gr.Button("新增/修改分類")
        add_output = gr.Markdown()
        add_btn.click(admin_add, [pwd, cat, lim], add_output)
        gr.Markdown("---")
        recalc_btn = gr.Button("重新計算所有分類支出")
        recalc_output = gr.Markdown()
        recalc_btn.click(admin_recalc, [pwd], recalc_output)




demo.launch(share=True)

✅ Gemini Client 初始化成功，已使用 Colab Secret 金鑰。


Exception ignored in: <function Client.__del__ at 0x7f9305e2fec0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/genai/client.py", line 400, in __del__
    self.close()
  File "/usr/local/lib/python3.12/dist-packages/google/genai/client.py", line 386, in close
    self._api_client.close()
    ^^^^^^^^^^^^^^^^
AttributeError: 'Client' object has no attribute '_api_client'


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://df17b0ba494a981ca0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
